In [429]:
import numpy as np
import random

In [420]:
ra = [7, 12, 17, 22,
      5,  9, 14, 20,
      4, 11, 16, 23,
      6, 10, 15, 21]
ra = [x % 8 for x in rotate_amounts]

constants = [int(abs(np.sin(4*i+4)) * 2**8) & 0xFF for i in range(16)]

init_values = [0x67, 0xef, 0x98, 0x10]

functions = 4*[lambda b, c, d: (b & c) | (~b & d)] + \
            4*[lambda b, c, d: (d & b) | (~d & c)] + \
            4*[lambda b, c, d: b ^ c ^ d] + \
            4*[lambda b, c, d: c ^ (b | ~d)]
 
index_functions = 4*[lambda i: i] + \
                  4*[lambda i: (5*i + 1)%4] + \
                  4*[lambda i: (3*i + 5)%4] + \
                  4*[lambda i: (7*i)%4]
 
def left_rotate(x, amount):
    x &= 0xFF
    return ((x<<amount) | (x>>(8-amount))) & 0xFF
 
def md5(message, rotate_amounts = ra):
 
    message = bytearray(message) #copy our input into a mutable buffer
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x8)
    while len(message)%16 != 8:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')
    #print(message, "added", orig_len_in_bits)
    #print(len(message))
 
    hash_pieces = init_values[:]
 
    for chunk_ofst in range(0, len(message), 16):
        a, b, c, d = hash_pieces
        chunk = message[chunk_ofst:chunk_ofst+16]
        for i in range(16):
            f = functions[i](b, c, d)
            g = index_functions[i](i)
            to_rotate = a + f + constants[i] + int.from_bytes(chunk[1*g:1*g+1], byteorder='little')
            new_b = (b + left_rotate(to_rotate, rotate_amounts[i])) & 0xFF
            a, b, c, d = d, new_b, b, c
        for i, val in enumerate([a, b, c, d]):
            hash_pieces[i] += val
            hash_pieces[i] &= 0xFF
 
    return sum(x<<(8*i) for i, x in enumerate(hash_pieces))
 
def md5_to_hex(digest):
    raw = digest.to_bytes(4, byteorder='little')
    return '{:8x}'.format(int.from_bytes(raw, byteorder='big'))


In [421]:
print(
    md5_to_hex(md5(b"AAAA")),
    md5_to_hex(md5(b"AAAC")),
    md5_to_hex(md5(b"AAAAA")),
    md5_to_hex(md5(b"AAAAB")),
    md5_to_hex(md5(b"ZAAAB")),
)

6079019b e6a1abf8 6079019b 6079019b 6a75a263


In [422]:
def find_best_rotate_amounts():
    collisions = []
    for i in range(100):
        rotate_amounts = [random.choice(range(1, 8)) for i in range(16)]
        collisions.append(find_collision(10001, 10200, rotate_amounts))
    return collisions

print(find_best_rotate_amounts())

[179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179, 179]


In [471]:
def find_collision(n = 1000, rotate_amounts = ra):
    hashes = {}
    for i in range(n):
        rnd_chars = [random.choice(range(0, 26)) for i in range(4)]
        string = "".join(chr(x + ord('A')) for x in rnd_chars)
        #hash_ = md5_to_hex(md5(bytes("{}".format(i), encoding='utf-8'), rotate_amounts))
        hash_ = md5_to_hex(md5(bytes(string, encoding='utf-8'), rotate_amounts))
        collisions = set()
        if hash_ in hashes:
            hashes[hash_].add(string)
            collisions.add(hash_)
        else:
            hashes[hash_] = set([string])
    return collisions, hashes

In [477]:
collisions, hashes = find_collision(1000000)

print(hashes[list(collisions)[0]])

{'TWDB'}


In [428]:
def find_n_zero_hash(n):
    hash_ = md5_to_hex(md5(b""))
    while(str(hash_)[-n:] != '0'*n):
        rnd_chars = [random.choice(range(0, 26)) for i in range(4)]
        string = "".join(chr(x + ord('A')) for x in rnd_chars)
        hash_ = md5_to_hex(md5(bytes(string, encoding='utf-8')))
    
    print("String:\t", string)
    print("Hash:\t", hash_)

find_n_zero_hash(4)

String:	 XEIA
Hash:	 7c610000
